<a href="https://colab.research.google.com/github/Angel-Castro-RC/Final_NLP/blob/main/F5_4_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 195: Natural Language Processing
## Embeddings

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericmanley/f23-CS195NLP/blob/main/F5_4_Embeddings.ipynb)


## References

Word2Vec Tutorial - The Skip-Gram Model by Chris McCormick: http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/

Word2Vec - Negative Sampling made easy by Munesh Lakhey: https://medium.com/@mnshonco/word2vec-negative-sampling-made-easy-9a587cb4695f

Keras Embedding Layer: https://keras.io/api/layers/core_layers/embedding/

Keras Tokenizer: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
import sys
!{sys.executable} -m pip install datasets keras tensorflow

## Dataset for today

AG News dataset
* short news articles
* four classes: World, Sports, Business, Sci/Tech

https://huggingface.co/datasets/ag_news


In [ ]:
from datasets import load_dataset
data = load_dataset("ag_news")

print(data["train"]["text"][0])

# 0 is World
# 1 is Sports
# 2 is Business
# 3 is Sci/Tech
print(data["train"]["label"][0])


Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
2


## Categorical classification example (with > 2 classes)

We've only seen binary classification examples so far
* used binary_crossentropy loss and a sigmoid output unit

For more than two classes, use categorical_crossentropy and softmax

In [ ]:
from datasets import load_dataset
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from datasets import load_dataset
import numpy as np


data = load_dataset("ag_news")
print("Here's an example of a sentence from the dataset:",data["train"]["text"][0])

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["train"]["text"])

# Convert text to sequence of integers
train_sequences = tokenizer.texts_to_sequences(data["train"]["text"])
test_sequences = tokenizer.texts_to_sequences(data["test"]["text"])
print("Here's an example of a tokenized sentence converted into a sequence of integers:",train_sequences[0])

# Pad sequences to ensure uniform length; you can decide the max length based on your dataset's characteristics
max_length = 100  # This should be adjusted based on the dataset
train_encoding_array = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_encoding_array = pad_sequences(test_sequences, maxlen=max_length, padding='post')
print("Here's an example after it has been padded:",train_encoding_array[0])


# Convert labels to one-hot vectors
train_labels = data["train"]["label"]
test_labels = data["test"]["label"]
train_labels_array = to_categorical(train_labels, num_classes=4)
test_labels_array = to_categorical(test_labels, num_classes=4)
print("Here's an example of what the target label looks like:", train_labels_array[0])


#create a neural network architecture
model = Sequential()
model.add(Dense(20, input_dim=max_length, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_encoding_array, train_labels_array, epochs=10, verbose=1)

loss, accuracy = model.evaluate(test_encoding_array, test_labels_array)
print(f"Test accuracy: {accuracy*100:.2f}%")

Here's an example of a sentence from the dataset: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
Here's an example of a tokenized sentence converted into a sequence of integers: [442, 441, 1681, 14528, 108, 64, 1, 850, 21, 21, 753, 8196, 442, 6640, 10231, 2927, 4, 5810, 25989, 40, 4049, 797, 332]
Here's an example after it has been padded: [  442   441  1681 14528   108    64     1   850    21    21   753  8196
   442  6640 10231  2927     4  5810 25989    40  4049   797   332     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0 

## Why does this do so badly?

No matter how many epochs you run this for, it will not get any better

**The problem:** the integer encoding

So what do we do if we still want to feed *sequential* data - we don't just want a bag of words?

## Word Embeddings

Don't just represent words with a single number - use a whole vector

<div>
   <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/embeddings.png?raw=1">
</div>

image source: https://stackoverflow.com/questions/46155868/keras-embedding-layer

in reality, you can't label the dimensions with exact meanings like "living being", "feline", etc.
* but you don't need to!

## How do we come up with these embeddings?

First, we'll come up with a "fake" learning problem and then extract information about words from the model

**fake learning problem** skip-grams: predict the words that appear around a given word

We can pick any window size - this one uses size 2 (2 words before and 2 after)


<div>
   <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/skip_gram_problem.png?raw=1">
</div>

image source: http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/

## One Hot Encoding

Before we can create a model for this, we need an initial numerical encoding of words

**One Hot Encoding** uses a vector with length equal to the size of the vocabulary
* 1 in the spot representing that word
* 0 in all others

|            | the | quick | brown | fox | jumps | over | lazy | dog |
|------------|-----|-----|-----|----|-----|-----|-----|-----|
| "fox" | 0   | 0   | 0   | 1  | 0   | 0   | 0   | 0   |
| "dog" | 0 | 0   | 0   | 0   | 0  | 0   | 0   | 1   |

We can use Keras' `to_categorical` for this

In [ ]:
from keras.utils import to_categorical

#put a 1 at index 3 of 8
print( to_categorical(3, num_classes=8)  )

[0. 0. 0. 1. 0. 0. 0. 0.]


## Let's get started

Here are some toy sentences we can work with.

We'll use Keras' tokenizer and show the mapping of words to indexes it cam up with

In [ ]:
from keras.preprocessing.text import Tokenizer

# Sample data
sentences = [
    "I adopted some dogs from the animal shelter",
    "don't you know that dogs and cats both like scritches",
    "are cats or dogs your favorite animal",
    "I have heard that dogs can be obedient",
    "I have heard that cats can be independent",
    "sharks live in the ocean",
    "many birds fly to get around"
]

# Tokenize and create vocabulary
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)


{'dogs': 1, 'i': 2, 'that': 3, 'cats': 4, 'the': 5, 'animal': 6, 'have': 7, 'heard': 8, 'can': 9, 'be': 10, 'adopted': 11, 'some': 12, 'from': 13, 'shelter': 14, "don't": 15, 'you': 16, 'know': 17, 'and': 18, 'both': 19, 'like': 20, 'scritches': 21, 'are': 22, 'or': 23, 'your': 24, 'favorite': 25, 'obedient': 26, 'independent': 27, 'sharks': 28, 'live': 29, 'in': 30, 'ocean': 31, 'many': 32, 'birds': 33, 'fly': 34, 'to': 35, 'get': 36, 'around': 37}


### So now we know all the words in our vocabulary

0 will be a special index to represent "None"

In [ ]:
vocabulary_size = len(tokenizer.word_index) + 1 #we also have to account for 0
print(vocabulary_size)

38


### Now we tokenize the sentences and convert them to their indexes

In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

[[2, 11, 12, 1, 13, 5, 6, 14], [15, 16, 17, 3, 1, 18, 4, 19, 20, 21], [22, 4, 23, 1, 24, 25, 6], [2, 7, 8, 3, 1, 9, 10, 26], [2, 7, 8, 3, 4, 9, 10, 27], [28, 29, 30, 5, 31], [32, 33, 34, 35, 36, 37]]


### Creating the skip grams

Keras has a nice function for this too.

In [ ]:
from keras.preprocessing.sequence import skipgrams

sequence_skipgrams = skipgrams(sequences[0],vocabulary_size=vocabulary_size,window_size=3)
print(sequence_skipgrams)

([[1, 11], [2, 1], [5, 6], [1, 5], [12, 25], [2, 33], [2, 11], [14, 5], [13, 6], [11, 22], [13, 14], [12, 15], [13, 16], [13, 10], [5, 36], [6, 28], [2, 12], [13, 1], [6, 5], [5, 25], [14, 21], [14, 6], [12, 17], [1, 13], [6, 22], [5, 37], [11, 29], [1, 13], [13, 16], [5, 13], [11, 1], [5, 12], [13, 21], [1, 21], [12, 36], [5, 14], [1, 36], [13, 28], [1, 30], [12, 5], [1, 2], [11, 23], [5, 35], [1, 12], [14, 11], [12, 2], [1, 6], [12, 13], [13, 30], [12, 1], [13, 12], [6, 5], [11, 19], [6, 32], [6, 13], [11, 13], [11, 12], [14, 13], [13, 5], [6, 1], [2, 13], [5, 1], [2, 19], [12, 11], [14, 22], [1, 25], [1, 31], [6, 14], [11, 2], [13, 11], [12, 1], [5, 16]], [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0])


### Group Discussion

What does this output mean? context word

is the starting word and the second shows the context word and is taking random words and util it find a pair that is in the window_size of three that is on the it will return 1 if not it will return 0

----
What are the different pairs of numbers?

The different pairs of numbers in the output represent skip-gram pairs generated from the input sentences.

----
What are all the 0s and 1s at the end?

The 1 are the right pairs that are connected and are in the same window size and the 0 are the pairs that are not right because of the window size

---

The `skipgrams` function is doing something called **negative sampling**. Write your guess for what that means here.

**Negative sampling:**

### Group Exercise: Prepare data for the skip-gram model

use `sequence_skipgrams` to prepare the data

Each training example should be the one hot encoded word concatenated with the one hot encoded context word

Example: if "brown" is index 3 and "fox" is index 4 (and vocab size is 8), then we have

"brown":`[0,0,0,1,0,0,0,0]`

"fox":`[0,0,0,0,1,0,0,0]`

training example input: `[0,0,0,1,0,0,0,0, 0,0,0,0,1,0,0,0]`

*Hint:* you will need to use the `to_categorical` function

Then, do it for all the skipgrams

In [ ]:
from keras.utils import to_categorical

list_of_inputs = []


for sequence in sequence_skipgrams[0]:
  first = to_categorical(sequence_skipgrams[0], num_classes=vocabulary_size)
  second = to_categorical(sequence_skipgrams[1], num_classes=vocabulary_size)
list_of_inputs.append(np.append(first, second))

list_of_inputs
array_of_inputs = np.array(list_of_inputs)
print(array_of_inputs)

target_array = np.array(sequence_skipgrams[1])

print(target_array)



[[0. 1. 0. ... 0. 0. 0.]]
[1 1 1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0
 0 0 1 1 0 0 1 0 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 1 0]


## A simple model

Let's draw what this model looks like on the white board

Then we'll train it with the data we prepared

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Model
embedding_model = Sequential()
# we input a one-hot vector for the word and its context word, so vocabulary_size*2
embedding_model.add(Dense(50, input_dim=vocabulary_size*2, activation='relu'))
embedding_model.add(Dense(1, activation='sigmoid'))

embedding_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

embedding_model.fit(array_of_inputs, target_array, epochs=5000, verbose=0)


ValueError: ignored

## Where is the word embedding in all this?

The weights going from the word's index to hidden layer nodes represent what the model learned about this word, so we'll use those weights as the embedding

In [ ]:
word = "dogs"
print("Word:", word)
word_index = tokenizer.word_index[word]
print("index:",word_index)
weights = embedding_model.layers[0].get_weights()[0]
word_embedding = weights[word_index]
print(f"The embedding for the word '{word}' is: {word_embedding.flatten()}")



Word: dogs
index: 1
The embedding for the word 'dogs' is: [ 0.06334592 -0.108724    0.20078011  0.10528298  0.04649441  0.13454293
  0.19588809 -0.05854668  0.12559398 -0.14612609  0.18730386 -0.00881907
  0.11768116 -0.14538726 -0.1638194  -0.04256137 -0.0286801  -0.11191082
  0.173331    0.19956516  0.13815127  0.0461192   0.0695533  -0.13321784
  0.1303504   0.06305854  0.02716325  0.08545946  0.05571072 -0.07696639
 -0.19211112  0.17094775 -0.20931146 -0.11042742  0.21601541 -0.02948251
 -0.03970143 -0.07031794  0.09496884  0.06523044 -0.10446328 -0.17345437
  0.04710193 -0.05973539  0.00672477 -0.06206429 -0.123109   -0.2154129
  0.19230102  0.11682345]


### We could make this into a function

In [ ]:
def get_embedding(word,embedding_model):
    word_index = tokenizer.word_index[word]
    weights = embedding_model.layers[0].get_weights()[0]
    word_embedding = weights[word_index]
    return word_embedding

cats_embedding = get_embedding("cats",embedding_model)
dogs_embedding = get_embedding("dogs",embedding_model)
shelter_embedding = get_embedding("shelter",embedding_model)
sharks_embedding = get_embedding("sharks",embedding_model)
print(cats_embedding)
print(dogs_embedding)

print( np.sum(np.square(cats_embedding-dogs_embedding)) )
print( np.sum(np.square(sharks_embedding-shelter_embedding)) )

[-0.20693704  0.12939014  0.18394561  0.1694359   0.04018794  0.12417655
 -0.15116742  0.10555975  0.02955098  0.09914751  0.08434899  0.20980914
  0.05336617 -0.09284978  0.09387995 -0.20582038 -0.19991134 -0.16432792
 -0.1543527   0.10696463 -0.12418482  0.19782819 -0.18390892  0.00321054
 -0.19298299 -0.07615934 -0.06046216  0.01445472 -0.12892585  0.10421427
 -0.01377998  0.03482066  0.15330152 -0.00800522 -0.17355269 -0.15042603
  0.09924479 -0.20494732 -0.00212656 -0.04873164 -0.17826335 -0.06276359
 -0.08834344 -0.01362333  0.15531279 -0.02217191 -0.11467648  0.12887596
 -0.01462859  0.08010806]
[-0.12367807 -0.14128283  0.14506651 -0.02959365 -0.16135804  0.14895882
 -0.01905528 -0.10689279  0.18205436 -0.18068844 -0.04844175  0.17377017
 -0.09151106  0.16277386 -0.06675974 -0.18409877 -0.15047988 -0.0447873
  0.05069451 -0.19084816  0.07452919  0.0899003   0.15209834 -0.13899466
  0.16538136  0.0185657   0.05095513 -0.14416815 -0.18711275  0.20634936
 -0.00358577  0.12141894  

## Applied Exploration

Create word embeddings for the AG News dataset.

Put all of the code into one cell so it isn't spread all throughout the notebook.

Show some example word embeddings.

Describe your results and reflect on them
* How many unique words does the dataset have?
* How many training epochs do you think are appropriate? Why?
* How could you go about figuring out if these embeddings are useful?



In [ ]:
from datasets import load_dataset
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Dense
from keras.layers import GlobalAveragePooling1D

# Load AG News dataset
data = load_dataset("ag_news")

# Tokenize and convert text to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["train"]["text"])
train_sequences = tokenizer.texts_to_sequences(data["train"]["text"])
test_sequences = tokenizer.texts_to_sequences(data["test"]["text"])

# Pad sequences for uniform length
max_length = 100
train_encoding_array = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_encoding_array = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Convert labels to one-hot vectors
num_classes = 4
train_labels_array = to_categorical(data["train"]["label"], num_classes=num_classes)
test_labels_array = to_categorical(data["test"]["label"], num_classes=num_classes)

# Create word embeddings using an Embedding layer in the neural network
vocabulary_size = len(tokenizer.word_index) + 1
embedding_dim = 50  # You can adjust the embedding dimension based on your dataset and task

model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=max_length))
model.add(GlobalAveragePooling1D())  # Global average pooling layer
model.add(Dense(20, activation='relu'))  # Optional additional dense layer
model.add(Dense(num_classes, activation='softmax'))  # Output layer with num_classes units for classification

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
epochs = 10
model.fit(train_encoding_array, train_labels_array, epochs=epochs, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(test_encoding_array, test_labels_array)
print(f"Test accuracy: {accuracy * 100:.2f}%")

# Example word embeddings
example_words = ["king", "queen", "man", "woman", "apple", "orange"]
word_index = tokenizer.word_index
for word in example_words:
    word_index = tokenizer.word_index[word]
    embedding_vector = model.layers[0].get_weights()[0][word_index]
    print(f"Word: {word}, Embedding: {embedding_vector}")

# Number of unique words in the dataset
unique_words = len(tokenizer.word_index)
print(f"Number of unique words in the dataset: {unique_words}")

# Number of training epochs
print(f"Number of training epochs: {epochs}")

ValueError: ignored

In [ ]:
from datasets import load_dataset
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Dense
from keras.layers import GlobalAveragePooling1D

# Load AG News dataset
data = load_dataset("ag_news")

# Tokenize and convert text to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["train"]["text"])
train_sequences = tokenizer.texts_to_sequences(data["train"]["text"])
test_sequences = tokenizer.texts_to_sequences(data["test"]["text"])

# Pad sequences for uniform length
max_length = 100
train_encoding_array = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_encoding_array = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Convert labels to one-hot vectors
num_classes = 4
train_labels_array = to_categorical(data["train"]["label"], num_classes=num_classes)
test_labels_array = to_categorical(data["test"]["label"], num_classes=num_classes)

# Create word embeddings using an Embedding layer in the neural network
vocabulary_size = len(tokenizer.word_index) + 1
embedding_dim = 50  # You can adjust the embedding dimension based on your dataset and task

model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=max_length))
model.add(GlobalAveragePooling1D())  # Global average pooling layer
model.add(Dense(20, activation='relu'))  # Optional additional dense layer
model.add(Dense(num_classes, activation='softmax'))  # Output layer with num_classes units for classification

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
epochs = 10
model.fit(train_encoding_array, train_labels_array, epochs=epochs, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(test_encoding_array, test_labels_array)
print(f"Test accuracy: {accuracy * 100:.2f}%")

# Example word embeddings
example_words = ["king", "queen", "man", "woman", "apple", "orange"]
word_index = tokenizer.word_index
for word in example_words:
    word_index = tokenizer.word_index[word]
    embedding_vector = model.layers[0].get_weights()[0][word_index]
    print(f"Word: {word}, Embedding: {embedding_vector}")

# Number of unique words in the dataset
unique_words = len(tokenizer.word_index)
print(f"Number of unique words in the dataset: {unique_words}")

# Number of training epochs
print(f"Number of training epochs: {epochs}")

ValueError: ignored

## The Keras Embedding Layer

Keras provides an `Embedding` layer that you can put at the beginning of your network which allows you to learn the embeddings as part of the main training process.

Let's see how it does when we include this in our example from the beginning of this notebook.

In [ ]:
from datasets import load_dataset
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from datasets import load_dataset
import numpy as np


data = load_dataset("ag_news")

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["train"]["text"])
vocabulary_size = len(tokenizer.word_index) + 1

# Convert text to sequence of integers
train_sequences = tokenizer.texts_to_sequences(data["train"]["text"])
test_sequences = tokenizer.texts_to_sequences(data["test"]["text"])



# Pad sequences to ensure uniform length; you can decide the max length based on your dataset's characteristics
max_length = 100  # This should be adjusted based on the dataset
train_encoding_array = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_encoding_array = pad_sequences(test_sequences, maxlen=max_length, padding='post')


# Convert labels to one-hot vectors
train_labels = data["train"]["label"]
test_labels = data["test"]["label"]
train_labels_array = to_categorical(train_labels, num_classes=4)
test_labels_array = to_categorical(test_labels, num_classes=4)

#create a neural network architecture
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=max_length))
model.add(Flatten())
model.add(Dense(20, input_dim=max_length, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_encoding_array, train_labels_array, epochs=10, verbose=1)

loss, accuracy = model.evaluate(test_encoding_array, test_labels_array)
print(f"Test accuracy: {accuracy*100:.2f}%")


Epoch 1/10
 527/3750 [===>..........................] - ETA: 2:40 - loss: 0.6960 - accuracy: 0.7280

KeyboardInterrupt: ignored